<a href="https://colab.research.google.com/github/Astra4051/Vulnerzone_Detector/blob/main/Vulnerzone_Detector_CODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from google.colab import drive
import os
import cv2
import numpy as np

# GPU 설정
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

# 구글 드라이브 마운트
drive.mount('/content/drive')

# 데이터 경로 설정
data_dir = '/content/drive/MyDrive'  # 데이터셋 폴더 경로로 변경

In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# 모델 로드
model_path = '/content/drive/MyDrive/Vulnerzone Detector Model.h5'
loaded_model = tf.keras.models.load_model(model_path)

# 이미지 경로 설정
test_image_path = '/content/drive/MyDrive/TestImage.png'
image_size = (224, 224)

# 이미지 로드 및 전처리
image = load_img(test_image_path, target_size=image_size)
image_array = img_to_array(image) / 255.0  # 이미지를 [0, 1] 범위로 스케일링
image_array = np.expand_dims(image_array, axis=0)  # 배치 차원 추가

# 모델 예측
prediction = loaded_model.predict(image_array)

# 취약한 부분 표시 함수 정의
def mark_individual_vulnerabilities(image_path, prediction):
    image = cv2.imread(image_path)
    vulnerability_mask = cv2.resize(np.uint8(prediction[0] * 255), (image.shape[1], image.shape[0]))
    threshold = 0.8

    if np.max(prediction) >= threshold:
        # 취약한 부분을 표시할 마스크 생성
        marked_mask = (vulnerability_mask > threshold * 255).astype(np.uint8)

        # 각 취약한 부분마다 네모 박스 그리기
        contours, _ = cv2.findContours(marked_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        marked_image = image.copy()
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            color = (0, 0, 255)  # 빨간색
            thickness = 2
            marked_image = cv2.rectangle(marked_image, (x, y), (x + w, y + h), color, thickness)

        marked_image_path = '/content/drive/MyDrive/MarkedImages/marked_image.png'
        cv2.imwrite(marked_image_path, marked_image)
        print("Marked image saved at:", marked_image_path)
    else:
        print("No vulnerability detected.")

# 이미지에 취약한 부분 표시
mark_individual_vulnerabilities(test_image_path, prediction)